In [3]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score
from pandas import Series, DataFrame
import scipy
from scipy import stats
import math

In [4]:
%matplotlib inline

In [5]:
train = pd.read_csv("Dataset-football-train.txt", delimiter="\t")

In [93]:
train.head()

,ID,Date,Opponent,Is_Home_or_Away,Is_Opponent_in_AP25_Preseason,Media,Label
0,1,sept_15,Texas,Home,Out,NBC,Win
1,2,sept_15,Virginia,Away,Out,ABC,Win
2,3,sept_15,Georgia Tech,Home,In,NBC,Win
3,4,sept_15,UMass,Home,Out,NBC,Win
4,5,oct_15,Clemson,Away,In,ABC,Lose


In [6]:
test = pd.read_csv("Dataset-football-test.txt", delimiter="\t")

Now we create the functions for information gain, split information, and gain ratio.

In [13]:
from collections import defaultdict
def information_gain(column_x, column_y, yes_label, no_label):
    yes = 0
    no = 0
    for item in column_y:
        if item == yes_label:
            yes += 1
        else:
            no += 1
    try:
        hy1 = (-float(yes)/len(column_y))* math.log(float(yes)/len(column_y), 2)
    except:
        hy1=0
    try:
        hy2 = (-float(no)/len(column_y))*math.log(float(no)/len(column_y), 2)
    except:
        hy2=0
    H_Y = hy1 + hy2
    options = defaultdict(list)
    for index, item in enumerate(column_x):
        options[item].append(index)
     
    y_given_x = 0
    for item in options:
        y = 0
        n = 0
        for index in options[item]:
            if column_y[index] == yes_label:
                y += 1
            else: 
                n += 1
                
        try:
            y_given_x_yes = (float(y)/len(column_x))*(-float(y)/len(item))* (math.log(float(y)/len(item), 2))
        except:
            y_given_x_yes = 0
        try:
            y_given_x_no = (float(n)/len(column_x))*(-float(n)/len(item))* (math.log(float(n)/len(item), 2))
        except:
            y_given_x_no = 0
        y_given_x += y_given_x_yes + y_given_x_no
    information_gain = abs(H_Y - y_given_x)
    return information_gain

In [38]:
def split_info(column_x):
    splitinfo = 0
    unique_values = set()
    for item in column_x:
        unique_values.add(item)
    for value in unique_values:
        count = 0
        for x in column_x:
            if value == x:
                count += 1
        try:
            splitinfo -= (float(count)/len(column_x)) * (math.log(float(count)/len(column_x), 2))
        except:
            continue
    return splitinfo 

In [53]:
def gain_ratio(informationgain, splitinformation):
    try:
        gain = float(informationgain)/splitinformation
    except:
        gain = 0
    print(gain)
    return

The dates are exact, so I decided to sort them into buckets based on month and year. This way if we actually split on date it is information that might be helpful in predicting. Otherwise, since there's no way the team would play two games on one day, the date would never be useful.

In [10]:
i = 0
for item in train["Date"]:
    if item[0] == "9" and item[len(item)-1] =="5":
        train["Date"][i] = "sept_15"
    elif item[0] == "9" and item[len(item)-1] =="6":
        train["Date"][i] = "sept_16"
    elif item[1] == "0" and item[len(item)-1] =="5":
        train["Date"][i] = "oct_15"
    elif item[1] == "0" and item[len(item)-1] =="6":
        train["Date"][i] = "oct_16"
    elif item[1] == "1" and item[len(item)-1] =="5":
        train["Date"][i] = "nov_15"
    else:
        train["Date"][i] = "nov_16"
    i += 1

/Users/kailee/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/kailee/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/kailee/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/kailee/anaconda/envs/py36

I turn all of the feature columns into lists so that they are in the proper format to go into my information gain function.

In [11]:
Label = list(train["Label"])
Media = list(train["Media"])
H_A = list(train["Is_Home_or_Away"])
Date = list(train["Date"])
Preseason = list(train["Is_Opponent_in_AP25_Preseason"])
Opponent = list(train["Opponent"])

Now we will find the gain ratio for all the features.

In [39]:
print("GR for Media is:")
gain_ratio(information_gain(Media, Label, "Win", "Lose"), split_info(Media))
print("GR for H_A is:")
gain_ratio(information_gain(H_A, Label, "Win", "Lose"), split_info(H_A))
print("GR for Date is:")
gain_ratio(information_gain(Date, Label, "Win", "Lose"), split_info(Date))
print("GR for Preseason is:")
gain_ratio(information_gain(Preseason, Label, "Win", "Lose"), split_info(Preseason))
print("GR for Opponent is:")
gain_ratio(information_gain(Opponent, Label, "Win", "Lose"), split_info(Opponent))

GR for Media is:
2.2728038661523553
GR for H_A is:
2.6450734821292787
GR for Date is:
0.19008987542825642
GR for Preseason is:
6.802739132585862
GR for Opponent is:
0.1333979174226363


Preseason has the highest gain ratio, so that is what we will branch on. This means that we need to sort all our other values into either branch 1 or branch 2 depending on whether they were in or out of the preseason.

In [40]:
Media1 = []
H_A1 = []
Date1 = []
Opponent1 = []
Label1 = []
Media2 = []
H_A2 = []
Date2 = []
Opponent2 = []
Label2 = []
for index, item in enumerate(Preseason):
    if item == "In":
        Media1.append(Media[index])
        H_A1.append(H_A[index])
        Date1.append(Date[index])
        Opponent1.append(Opponent[index])
        Label1.append(Label[index])
    else:
        Media2.append(Media[index])
        H_A2.append(H_A[index])
        Date2.append(Date[index])
        Opponent2.append(Opponent[index])
        Label2.append(Label[index])
In = pd.DataFrame()
Out = pd.DataFrame()
In["Media"] = Media1
In["H_A"] = H_A1
In["Date"] = Date1
In["Opponent"] = Opponent1
In["Label"] = Label1
Out["Media"] = Media2
Out["H_A"] = H_A2
Out["Date"] = Date2
Out["Opponent"] = Opponent2
Out["Label"] = Label2

In [41]:
In.head()

,Media,H_A,Date,Opponent,Label
0,NBC,Home,sept_15,Georgia Tech,Win
1,ABC,Away,oct_15,Clemson,Lose
2,NBC,Home,oct_15,USC,Win
3,FOX,Away,nov_15,Stanford,Lose
4,NBC,Home,oct_16,Stanford,Lose


In [46]:
print("GR for Media In Preseason is:")
gain_ratio(information_gain(Media1, Label1, "Win", "Lose"), split_info(Media1))
print("GR for H_A In Preseason is:")
gain_ratio(information_gain(H_A1, Label1, "Win", "Lose"), split_info(H_A1))
print("GR for Date In Preseason is:")
gain_ratio(information_gain(Date1, Label1, "Win", "Lose"), split_info(Date1))
print("GR for Opponent In Preseason is:")
gain_ratio(information_gain(Opponent1, Label1, "Win", "Lose"), split_info(Opponent1))

GR for Media In Preseason is:
0.3288274626044878
GR for H_A In Preseason is:
0.4506584375036809
GR for Date In Preseason is:
0.18278414959221004
GR for Opponent In Preseason is:
0.1917233815272914


We will split on Home/Away since it is the highest gain ratio.

In [44]:
Media_home = []
Date_home = []
Opponent_home = []
Label_home = []
Media_away = []
Date_away = []
Opponent_away = []
Label_away = []
for index, item in enumerate(H_A1):
    if item == "Home":
        Media_home.append(Media1[index])
        Date_home.append(Date1[index])
        Opponent_home.append(Opponent1[index])
        Label_home.append(Label1[index])
    else:
        Media_away.append(Media1[index])
        Date_away.append(Date1[index])
        Opponent_away.append(Opponent1[index])
        Label_away.append(Label1[index])

In [54]:
print("GR for Media in In_Home is:")
gain_ratio(information_gain(Media_home, Label_home, "Win", "Lose"), split_info(Media_home))
print("GR for Date in In_Home is:")
gain_ratio(information_gain(Date_home, Label_home, "Win", "Lose"), split_info(Date_home))
print("GR for Opponent in In_Home is:")
gain_ratio(information_gain(Opponent_home, Label_home, "Win", "Lose"), split_info(Opponent_home))

GR for Media in In_Home is:
0
GR for Date in In_Home is:
0.28830850581104195
GR for Opponent in In_Home is:
0.31416038528480106


The highest gain ratio is Opponent, so we will split on that. Now we check for Away.

In [56]:
print("GR for Media in In_Away is:")
gain_ratio(information_gain(Media_away, Label_away, "Win", "Lose"), split_info(Media_away))
print("GR for Date in In_Away is:")
gain_ratio(information_gain(Date_away, Label_away, "Win", "Lose"), split_info(Date_away))
print("GR for Opponent in In_Away is:")
gain_ratio(information_gain(Opponent_away, Label_away, "Win", "Lose"), split_info(Opponent_away))

GR for Media in In_Away is:
0.4748908380134757
GR for Date in In_Away is:
0.27182162559524287
GR for Opponent in In_Away is:
0.27432227074380156


The highest gain ratio is Media so we will split on that. But first, we return to the Out branch.

In [45]:
Out.head()

,Media,H_A,Date,Opponent,Label
0,NBC,Home,sept_15,Texas,Win
1,ABC,Away,sept_15,Virginia,Win
2,NBC,Home,sept_15,UMass,Win
3,NBC,Home,oct_15,Navy,Win
4,ABC,Away,oct_15,Temple,Win


In [47]:
print("GR for Media in Out Preseason is:")
gain_ratio(information_gain(Media2, Label2, "Win", "Lose"), split_info(Media2))
print("GR for H_A in Out Preseason is:")
gain_ratio(information_gain(H_A2, Label2, "Win", "Lose"), split_info(H_A2))
print("GR for Date in Out Preseason is:")
gain_ratio(information_gain(Date2, Label2, "Win", "Lose"), split_info(Date2))
print("GR for Opponent in Out Preseason is:")
gain_ratio(information_gain(Opponent2, Label2, "Win", "Lose"), split_info(Opponent2))

GR for Media in Out Preseason is:
1.7218353111111935
GR for H_A in Out Preseason is:
1.816442622529741
GR for Date in Out Preseason is:
0.1482471247085744
GR for Opponent in Out Preseason is:
0.12206599213447909


The highest gain ratio is Home/Away, so we will split on that.

In [48]:
Media_home2 = []
Date_home2 = []
Opponent_home2 = []
Label_home2 = []
Media_away2 = []
Date_away2 = []
Opponent_away2 = []
Label_away2 = []
for index, item in enumerate(H_A2):
    if item == "Home":
        Media_home2.append(Media2[index])
        Date_home2.append(Date2[index])
        Opponent_home2.append(Opponent2[index])
        Label_home2.append(Label2[index])
    else:
        Media_away2.append(Media2[index])
        Date_away2.append(Date2[index])
        Opponent_away2.append(Opponent2[index])
        Label_away2.append(Label2[index])

In [49]:
print("GR for Media in Out_Home is:")
gain_ratio(information_gain(Media_home2, Label_home2, "Win", "Lose"), split_info(Media_home2))
print("GR for Date in Out_Home is:")
gain_ratio(information_gain(Date_home2, Label_home2, "Win", "Lose"), split_info(Date_home2))
print("GR for Opponent in Out_Home is:")
gain_ratio(information_gain(Opponent_home2, Label_home2, "Win", "Lose"), split_info(Opponent_home2))

GR for Media in Out_Home is:
2.8827292687087667
GR for Date in Out_Home is:
0.14844250168055695
GR for Opponent in Out_Home is:
0.1301713511509548


The highest gain ratio is Media so we will split on that. But first we return to the Away branch to check the gain ratios there.

In [51]:
print("GR for Media in Out_Away is:")
gain_ratio(information_gain(Media_away2, Label_away2, "Win", "Lose"), split_info(Media_away2))
print("GR for Date in Out_Away is:")
gain_ratio(information_gain(Date_away2, Label_away2, "Win", "Lose"), split_info(Date_away2))
print("GR for Opponent in Out_Away is:")
gain_ratio(information_gain(Opponent_away2, Label_away2, "Win", "Lose"), split_info(Opponent_away2))

GR for Media in Out_Away is:
1.0772716011696248
GR for Date in Out_Away is:
0.20647107469050285
GR for Opponent in Out_Away is:
0.20964275582594497


The highest gain ratio is Media so we will split on that. 
For now though, we consider the In branch and the splits that we have determined must occur there. For In_Home, the gain ratio was highest for Opponent, but since there are no repeat opponents in that list, we will not need to calculate gain ratio for those branches, as there will be nothing left to split on. For In_Away, the highest gain ratio was Media, and there are two different branches that must be created, one for ABC and the other for FOX. However, all of the games in In_Away were loses, meaning we do not actually need to branch. So, the In side of the tree is complete.
We look at the Out side again. We are splitting on Media for both Out_Home and Out_Away, and so we must sort our games into the correct lists. First we will sort for Home.

In [80]:
Date_NBC = []
Opponent_NBC = []
Label_NBC = []
Date_ESPN = []
Opponent_ESPN = []
Label_ESPN = []
Date_CBS = []
Opponent_CBS = []
Label_CBS = []
for index, item in enumerate(Media_home2):
    if item == "NBC":
        Date_NBC.append(Date_home2[index])
        Opponent_NBC.append(Opponent_home2[index])
        Label_NBC.append(Label_home2[index])
    elif item == "ESPN":
        Date_ESPN.append(Date_home2[index])
        Opponent_ESPN.append(Opponent_home2[index])
        Label_ESPN.append(Label_home2[index])
    else:
        Date_CBS.append(Date_home2[index])
        Opponent_CBS.append(Opponent_home2[index])
        Label_CBS.append(Label_home2[index])

Since ESPN and CBS only have one game each, we will not need to calculate gain ratio for those or split. So we only calculate the gain ratio for NBC.

In [81]:
print("GR for Date in Out_Home_NBC is:")
gain_ratio(information_gain(Date_NBC, Label_NBC, "Win", "Lose"), split_info(Date_NBC))
print("GR for Opponent in Out_Home_NBC is:")
gain_ratio(information_gain(Opponent_NBC, Label_NBC, "Win", "Lose"), split_info(Opponent_NBC))

GR for Date in Out_Home_NBC is:
0.12344443055879188
GR for Opponent in Out_Home_NBC is:
0.11031264567713367


The highest gain ratio is Date, so we will split on that. First we sort Out_Away for the Media branch though.

In [82]:
Date_NBC2 = []
Opponent_NBC2 = []
Label_NBC2 = []
Date_ABC2 = []
Opponent_ABC2 = []
Label_ABC2 = []
for index, item in enumerate(Media_away2):
    if item == "NBC":
        Date_NBC2.append(Date_away2[index])
        Opponent_NBC2.append(Opponent_away2[index])
        Label_NBC2.append(Label_away2[index])
    else:
        Date_ABC2.append(Date_away2[index])
        Opponent_ABC2.append(Opponent_away2[index])
        Label_ABC2.append(Label_away2[index])

Since there is only one game in NBC2, we do not need to calculate gain ratio for that, since there is nothing else to split on. So we only calculate it for ABC2.

In [83]:
print("GR for Date in Out_Away_ABC is:")
gain_ratio(information_gain(Date_ABC2, Label_ABC2, "Win", "Lose"), split_info(Date_ABC2))
print("GR for Opponent in Out_Away_ABC is:")
gain_ratio(information_gain(Opponent_ABC2, Label_ABC2, "Win", "Lose"), split_info(Opponent_ABC2))

GR for Date in Out_Away_ABC is:
0.23774813024470418
GR for Opponent in Out_Away_ABC is:
0.24707430616942733


The highest gain ratio is Opponent, so we will split on that. However, since there are no repeat opponents in this list, then we are done for this branch. So now we go back to Out_Home_NBC and split on Date. We only need to create lists for branches that have more than one game, that is, sept15 and sept16.

In [92]:
Date_NBC

['sept_15',
 'sept_15',
 'oct_15',
 'nov_15',
 'sept_16',
 'sept_16',
 'sept_16',
 'oct_16',
 'nov_16']

In [87]:
Opponent_sept15 = []
Label_sept15 = []
Opponent_sept16 = []
Label_sept16 = []
for index, item in enumerate(Date_NBC):
    if item == "sept_15":
        Opponent_sept15.append(Opponent_NBC[index])
        Label_sept15.append(Label_NBC[index])
    else:
        Opponent_sept16.append(Opponent_NBC[index])
        Label_sept16.append(Label_NBC[index])

In the sept15 list we only have wins, which means we are done. For the sept16 list since we have some wins and some losses, we will have to split on the last feature Opponents, but these are all individual games, so we are also done. Now the tree is complete! Please see the PDF file for a drawing. 

Now we can construct a function to test our test data based on our constructed tree.

In [88]:
Label_sept15

['Win', 'Win']

In [90]:
Opponent_sept16

['Navy',
 'Wake Forest',
 'Nevada',
 'Michigan State',
 'Duke',
 'Miami Florida',
 'Army']

In [116]:
def predict(dataframe):
    predictions = []
    for index, row in dataframe.iterrows():
        if row["Is_Opponent_in_AP25_Preseason"] == "In":
            if row["Is_Home_or_Away"] == "Home":
                if row["Opponent"] == "Stanford" or "Virginia Tech":
                    predictions.append("Lose")
                else:
                    predictions.append("Win")
            else:
                predictions.append("Lose")
        else:
            if row["Is_Home_or_Away"] == "Home":
                if row["Media"] == "CBS":
                    predictions.append("Lose")
                elif row["Media"] == "ESPN":
                    predictions.append("Win")
                elif row["Media"] == "NBC":
                    if row["Date"] == "sept_16":
                        if row["Opponent"] == "Nevada":
                            predictions.append("Win")
                        else:
                            predictions.append("Lose")
                    else:
                        predictions.append("Win")
                else:
                    predictions.append("Win")
            else:
                if row["Media"] == "ABC":
                    if row["Opponent"] == "North Carolina State" or "Texas":
                        predictions.append("Lose")
                    else:
                        predictions.append("Win")
                else:
                    predictions.append("Win")
    return predictions

Now we do the same Date preprocessing for the test set that we did on the train set.

In [95]:
i = 0
for item in test["Date"]:
    print(item)
    if item[0] == "9" and item[len(item)-1] =="7":
        test["Date"][i] = "sept_17"
    elif item[1] == "0" and item[len(item)-1] =="7":
        test["Date"][i] = "oct_17"
    else:
        test["Date"][i] = "nov_17"
    i += 1

/Users/kailee/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [108]:
test.head()

,ID,Date,Opponent,Is_Home_or_Away,Is_Opponent_in_AP25_Preseason,Media,Label
0,25,nov_16,Temple,Home,Out,NBC,Win
1,26,nov_16,Georgia,Home,In,NBC,Lose
2,27,nov_16,Boston College,Away,Out,ESPN,Win
3,28,nov_16,Michigan State,Away,Out,FOX,Win
4,29,nov_16,Miami Ohio,Home,Out,NBC,Win


In [117]:
predictions = predict(test)

In [118]:
print(predictions)

['Win', 'Lose', 'Win', 'Win', 'Win', 'Lose', 'Lose', 'Win', 'Win', 'Lose', 'Win', 'Lose']


In [119]:
actual = list(test["Label"])

In [121]:
precision_score(actual, predictions, pos_label="Win")

1.0

In [122]:
recall_score(actual, predictions, pos_label="Win")

0.7777777777777778

In [123]:
f1_score(actual, predictions, pos_label="Win")

0.8750000000000001